Impoting necessary libararies

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Lambda, Dropout # Import Dropout here
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Loading data from folders

In [ ]:
def load_data_from_folder(folder_path, label):
    matrices = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, skiprows=1, header=None)
            df = df.iloc[:, 1:]
            matrix = df.values.astype(np.float32)
            matrices.append(matrix)
            labels.append(label)
    return np.array(matrices), np.array(labels)

In [ ]:
adhd_folder = "/content/adhd"
control_folder = "/content/control"
adhd_data, adhd_labels = load_data_from_folder(adhd_folder, label=1)
control_data, control_labels = load_data_from_folder(control_folder, label=0)

Combining and splitting the data

In [ ]:
X = np.concatenate([adhd_data, control_data])
y = np.concatenate([adhd_labels, control_labels])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)


Reshaping the data

In [ ]:
X_train = X_train.reshape(-1, 19, 19, 1)
X_val = X_val.reshape(-1, 19, 19, 1)
X_test = X_test.reshape(-1, 19, 19, 1)

Base network

In [ ]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

In [ ]:
def create_base_network(input_shape):
    model = Sequential()
    model.add(RandomFlip("horizontal"))
    model.add(RandomRotation(0.2))
    model.add(RandomZoom(0.2))
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    # Additional layers for higher feature extraction capacity
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.6))
    return model

In [ ]:
input_shape = (19, 19, 1)
base_network = create_base_network(input_shape)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Defining inputs to generate feature vectors

In [ ]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

feat_vecs_a = base_network(input_a)
feat_vecs_b = base_network(input_b)

Calculating the distance between two features

In [ ]:
distance = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]), output_shape=(128,))([feat_vecs_a, feat_vecs_b])
output = Dense(1, activation='sigmoid')(distance)
import tensorflow as tf


Triplet Loss

In [ ]:
def triplet_loss(margin):
    def loss(y_true, y_pred):
        anchor, positive, negative = y_pred[:, :128], y_pred[:, 128:256], y_pred[:, 256:]
        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
        loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
        return tf.reduce_mean(loss)
    return loss


Creating Triplets for the Siamese Network

In [ ]:
def create_triplets(X, y):
    triplets = []
    labels = []

    n = min(len(X[y == 0]), len(X[y == 1]))
    for i in range(n):
        anchor = X[y == 0][i]
        positive = X[y == 0][(i + 1) % n]

        neg_index = np.random.choice(np.where(y == 1)[0])
        negative = X[neg_index]

        triplets.append([anchor, positive, negative])
        labels.append(0)

    return np.array(triplets), np.array(labels)

Preparing training and validation triplets

In [ ]:
train_triplets, train_labels = create_triplets(X_train, y_train)
val_triplets, val_labels = create_triplets(X_val, y_val)

Defining the model using the base network for three inputs

In [ ]:
input_shape = (19, 19, 1)
base_network = create_base_network(input_shape)

input_anchor = Input(shape=input_shape)
input_positive = Input(shape=input_shape)
input_negative = Input(shape=input_shape)

feat_vecs_anchor = base_network(input_anchor)
feat_vecs_positive = base_network(input_positive)
feat_vecs_negative = base_network(input_negative)

Concatenating feature vectors for triplet loss calculation

In [ ]:
merged_output = tf.keras.layers.concatenate([feat_vecs_anchor, feat_vecs_positive, feat_vecs_negative], axis=1)

In [ ]:
# Defining and compiling the model
siamese_model = Model(inputs=[input_anchor, input_positive, input_negative], outputs=merged_output)
siamese_model.compile(loss=triplet_loss(margin=0.8), optimizer='adam', metrics=['accuracy'])


# Defining callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('siamese_model.keras', save_best_only=True, monitor='val_loss')

In [ ]:
history = siamese_model.fit(
    [train_triplets[:, 0], train_triplets[:, 1], train_triplets[:, 2]], train_labels,
    epochs=100,
    batch_size=25,
    validation_data=([val_triplets[:, 0], val_triplets[:, 1], val_triplets[:, 2]], val_labels),
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 434ms/step - accuracy: 0.0000e+00 - loss: 112.1931 - val_accuracy: 0.0000e+00 - val_loss: 0.7624
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.0000e+00 - loss: 116.7514 - val_accuracy: 0.0000e+00 - val_loss: 0.7616
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.0000e+00 - loss: 96.2832 - val_accuracy: 0.0000e+00 - val_loss: 0.7563
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.0000e+00 - loss: 84.9051 - val_accuracy: 0.0000e+00 - val_loss: 0.7484
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.0000e+00 - loss: 56.0571 - val_accuracy: 0.0000e+00 - val_loss: 0.7384
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.0000e+00 - loss: 65.4811 - val_accuracy: 0.0000e+00 - val_loss: 0.7338
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.0000e+00 - loss: 35.1172 - val_accuracy: 0.0000e+00 - val_loss: 0.7275
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


Function to compute distances

In [ ]:
def compute_distances(anchor, positive, negative):
    pos_dist = np.sum(np.square(anchor - positive), axis=1)
    neg_dist = np.sum(np.square(anchor - negative), axis=1)
    return pos_dist, neg_dist

Preparing test triplets to make predictions

In [ ]:
test_triplets, test_labels = create_triplets(X_test, y_test)


Predicting embeddings for all samples at once

In [ ]:
embeddings = siamese_model.predict([test_triplets[:, 0], test_triplets[:, 1], test_triplets[:, 2]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step


Spliting the embeddings into anchor, positive, and negative

In [ ]:

anchor_embeddings = embeddings[:, :128]
positive_embeddings = embeddings[:, 128:256]
negative_embeddings = embeddings[:, 256:]


Calculating the distances

In [ ]:
pos_dist, neg_dist = compute_distances(anchor_embeddings, positive_embeddings, negative_embeddings)


Determining predictions based on distances

In [ ]:
y_pred = (pos_dist < neg_dist).astype(int)

In [ ]:
print("Accuracy:", accuracy_score(test_labels, y_pred))
print(np.concatenate((test_labels.reshape(-1, 1), y_pred.reshape(-1,1)), axis=1))
print(confusion_matrix(test_labels, y_pred))
print(classification_report(test_labels, y_pred))

Accuracy: 0.5
[[0 0]
 [0 1]
 [0 1]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]]
[[6 6]
 [0 0]]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.00      0.00      0.00         0

    accuracy                           0.50        12
   macro avg       0.50      0.25      0.33        12
weighted avg       1.00      0.50      0.67        12



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
